In [ ]:
%matplotlib inline

In [ ]:
import pickle
import numpy as np
import awkward
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import uproot
import boost_histogram as bh
import mplhep
import glob
import os
import vector
import shutil

mplhep.style.use("CMS")

In [ ]:
import sys

sys.path += ["../../mlpf/plotting/"]
sys.path += ["../../mlpf/"]

import plot_utils
import jet_utils

In [ ]:
def to_bh(data, bins, cumulative=False):
    h1 = bh.Histogram(bh.axis.Variable(bins))
    h1.fill(data)
    if cumulative:
        h1[:] = np.sum(h1.values()) - np.cumsum(h1)
    return h1


def load_pickle(fn):
    d = pickle.load(open(fn, "rb"))
    ret = []
    for it in d:
        ret.append(
            {
                "slimmedGenJets": it["slimmedGenJets"],
                "slimmedJetsPuppi": it["slimmedJetsPuppi"],
                "genMetTrue": it["genMetTrue"],
                "slimmedMETsPuppi": it["slimmedMETsPuppi"],
            }
        )
    return ret


def varbins(*args):
    newlist = []
    for arg in args[:-1]:
        newlist.append(arg[:-1])
    newlist.append(args[-1])
    return np.concatenate(newlist)


def get_hist_and_merge(files, histname):
    hists = []
    for fn in files:
        fi = uproot.open(fn)
        h = fi[histname].to_boost()
        hists.append(h)
    return sum(hists[1:], hists[0])


from scipy.optimize import curve_fit


def Gauss(x, a, x0, sigma):
    return a * np.exp(-((x - x0) ** 2) / (2 * sigma**2))


def fit_response(hist2d, bin_range):
    centers = []
    means = []
    means_unc = []

    sigmas = []
    sigmas_unc = []

    for ibin in bin_range:

        print(ibin)
        plt.figure()
        xvals = hist2d.axes[1].centers
        vals = hist2d.values()[ibin]
        errs = np.sqrt(vals)
        errs[vals == 0] = 1.0

        parameters1, covariances1 = curve_fit(
            Gauss,
            xvals,
            vals,
            p0=[1.0, 0.0, 1.0],
            sigma=errs,
            maxfev=1000000,
            method="dogbox",
            bounds=[(-np.inf, -10, 0), (np.inf, 10, 50)],
        )
        plt.errorbar(xvals, vals, errs)
        plt.plot(xvals, Gauss(xvals, *parameters1))
        plt.xlabel("$\Delta E_T / E_T$")
        plt.title("${} < E_T < {}$".format(hist2d.axes[0].edges[ibin], hist2d.axes[0].edges[ibin + 1]))

        means.append(parameters1[1])
        means_unc.append(np.sqrt(covariances1[1, 1]))
        sigmas.append(parameters1[2])
        sigmas_unc.append(np.sqrt(covariances1[2, 2]))

        centers.append(hist2d.axes[0].centers[ibin])

    centers = np.array(centers)
    means = np.array(means)
    means_unc = np.array(means_unc)

    sigmas = np.array(sigmas)
    sigmas_unc = np.array(sigmas_unc)

    return centers, means, means_unc, sigmas, sigmas_unc

In [ ]:
from plot_utils import ELEM_LABELS_CMS, ELEM_NAMES_CMS
from plot_utils import CLASS_LABELS_CMS, CLASS_NAMES_CMS
from plot_utils import cms_label, sample_label

In [ ]:
folder = "TTbar_PU"

if folder == "QCD_PU":
    jet_bins = varbins(np.linspace(10, 100, 21), np.linspace(100, 200, 5), np.linspace(200, 1000, 5))
    met_bins = varbins(np.linspace(0, 150, 21), np.linspace(150, 500, 5))
    physics_process = "RelValQCD_FlatPt_15_3000HS_14"

if folder == "TTbar_PU":
    jet_bins = varbins(np.linspace(10, 100, 21), np.linspace(100, 250, 5))
    met_bins = varbins(np.linspace(0, 150, 21), np.linspace(150, 250, 5))
    physics_process = "RelValTTbar_14TeV"

outpath = "cmssw/{}".format(folder)
shutil.rmtree(outpath, ignore_errors=True)
os.makedirs(outpath)

In [ ]:
pf_files = glob.glob("/local/joosep/mlpf/results/cms/CMSSW_14_1_0_pre3/{}_pf/step3_MINI_*.pkl".format(folder))
mlpf_2022_files = glob.glob("/local/joosep/mlpf/results/cms/CMSSW_14_1_0_pre3/{}_mlpf_acat2022/step3_MINI_*.pkl".format(folder))
mlpf_files = glob.glob("/local/joosep/mlpf/results/cms/CMSSW_14_1_0_pre3/{}_mlpf/step3_MINI_*.pkl".format(folder))

pf_files_d = {os.path.basename(fn): fn for fn in pf_files}
mlpf_2022_files_d = {os.path.basename(fn): fn for fn in mlpf_2022_files}
mlpf_files_d = {os.path.basename(fn): fn for fn in mlpf_files}

In [ ]:
common_files = list(set(pf_files_d.keys()).intersection(set(mlpf_files_d.keys())))
common_files

In [ ]:
data_baseline = sum([load_pickle(pf_files_d[fn]) for fn in common_files], [])
data_mlpf_old = sum([load_pickle(mlpf_2022_files_d[fn]) for fn in common_files], [])
data_mlpf_new = sum([load_pickle(mlpf_files_d[fn]) for fn in common_files], [])

In [ ]:
def jet_vec(data, key):
    arr = awkward.from_iter([d[key] for d in data])
    jet_vec = vector.awk(awkward.zip({"pt": arr.pt, "eta": arr.eta, "phi": arr.phi, "energy": arr.energy}))
    return jet_vec

In [ ]:
gen_jets = jet_vec(data_baseline, "slimmedGenJets")
pf_jets = jet_vec(data_baseline, "slimmedJetsPuppi")
mlpf_old_jets = jet_vec(data_mlpf_old, "slimmedJetsPuppi")
mlpf_new_jets = jet_vec(data_mlpf_new, "slimmedJetsPuppi")

gen_met_pt = awkward.flatten(awkward.from_iter([d["genMetTrue"]["pt"] for d in data_baseline]))
pf_met_pt = awkward.flatten(awkward.from_iter([d["slimmedMETsPuppi"]["pt"] for d in data_baseline]))
mlpf_old_met_pt = awkward.flatten(awkward.from_iter([d["slimmedMETsPuppi"]["pt"] for d in data_mlpf_old]))
mlpf_new_met_pt = awkward.flatten(awkward.from_iter([d["slimmedMETsPuppi"]["pt"] for d in data_mlpf_new]))

In [ ]:
def match_jets(jet1, jet2, deltar):
    ind1, ind2 = jet_utils.match_jets(jet1, jet2, deltar)
    return {
        "pt_1": awkward.flatten(jet1[ind1].pt), 
        "eta_1": awkward.flatten(jet1[ind1].pt), 
        "pt_2": awkward.flatten(jet2[ind2].pt), 
        "eta_2": awkward.flatten(jet2[ind2].pt)
    }

In [ ]:
gen_pf_match = match_jets(gen_jets, pf_jets, 0.1)
gen_mlpf_old_match = match_jets(gen_jets, mlpf_old_jets, 0.1)
gen_mlpf_new_match = match_jets(gen_jets, mlpf_new_jets, 0.1)

In [ ]:
f, (a0, a1) = plt.subplots(2, 1, gridspec_kw={"height_ratios": [3, 1]}, sharex=True)

h0 = to_bh(awkward.flatten(gen_jets.pt), jet_bins)
h1 = to_bh(awkward.flatten(pf_jets.pt), jet_bins)
h2 = to_bh(awkward.flatten(mlpf_old_jets.pt), jet_bins)
h3 = to_bh(awkward.flatten(mlpf_new_jets.pt), jet_bins)

plt.sca(a0)
x0 = mplhep.histplot(h0, histtype="step", lw=2, label="gen", binwnorm=1.0, ls="--")
x1 = mplhep.histplot(h1, histtype="step", lw=2, label="PF", binwnorm=1.0, ls="-")
x2 = mplhep.histplot(h2, histtype="step", lw=2, label="MLPF old", binwnorm=1.0, ls="-")
x3 = mplhep.histplot(h3, histtype="step", lw=2, label="MLPF new", binwnorm=1.0, ls="-")

# plt.xscale("log")
plt.yscale("log")
cms_label(a0)
# sample_label(a0, physics_process)
a0.text(0.01, 0.92, "AK4 PUPPI jets", transform=a0.transAxes)
handles, labels = a0.get_legend_handles_labels()
handles = [x0[0].stairs, x1[0].stairs, x2[0].stairs, x3[0].stairs]
a0.legend(handles, labels, loc=1)
plt.ylim(10, 10**6)
plt.ylabel("Number of jets / GeV")

plt.sca(a1)
mplhep.histplot(h0 / h0, histtype="step", lw=2, ls="--")
mplhep.histplot(h1 / h0, histtype="step", lw=2, ls="-")
mplhep.histplot(h2 / h0, histtype="step", lw=2, ls="-")
mplhep.histplot(h3 / h0, histtype="step", lw=2, ls="-")
plt.ylim(0.5,1.5)
plt.ylabel("reco / gen")
plt.xlabel("jet $p_T$ [GeV]")

plt.xscale("log")

plt.xlim(min(jet_bins), max(jet_bins))
plt.savefig("{}/ak4_puppi_jet_pt.pdf".format(outpath))

In [ ]:
import scipy
import scipy.stats

In [ ]:
def compute_iqr(data):
    p75 = np.percentile(data, 75)
    p25 = np.percentile(data, 25)
    return p75-p25

In [ ]:
b = np.linspace(0, 2, 100)

med_vals_pf = []
med_vals_mlpf_old = []
med_vals_mlpf_new = []

iqr_vals_pf = []
iqr_vals_pf_low = []
iqr_vals_pf_high = []

iqr_vals_mlpf_old = []
iqr_vals_mlpf_old_low = []
iqr_vals_mlpf_old_high = []

iqr_vals_mlpf_new = []
iqr_vals_mlpf_new_low = []
iqr_vals_mlpf_new_high = []

for ibin in range(len(jet_bins)-1):
    min_pt = jet_bins[ibin]
    max_pt = jet_bins[ibin+1]

    response_pf = (gen_pf_match["pt_2"] / gen_pf_match["pt_1"])[(gen_pf_match["pt_1"]>=min_pt) & (gen_pf_match["pt_1"]<max_pt)]
    response_mlpf_old = (gen_mlpf_old_match["pt_2"] / gen_mlpf_old_match["pt_1"])[(gen_mlpf_old_match["pt_1"]>=min_pt) & (gen_mlpf_old_match["pt_1"]<max_pt)]
    response_mlpf_new = (gen_mlpf_new_match["pt_2"] / gen_mlpf_new_match["pt_1"])[(gen_mlpf_new_match["pt_1"]>=min_pt) & (gen_mlpf_new_match["pt_1"]<max_pt)]

    med, iqr = plot_utils.med_iqr(response_pf)
    med_vals_pf.append(med)
    iqr_vals_pf.append(iqr)

    bsres = scipy.stats.bootstrap((awkward.to_numpy(response_pf), ), compute_iqr)
    iqr_vals_pf_low.append(bsres.confidence_interval.low)
    iqr_vals_pf_high.append(bsres.confidence_interval.high)

    bsres = scipy.stats.bootstrap((awkward.to_numpy(response_mlpf_old), ), compute_iqr)
    iqr_vals_mlpf_old_low.append(bsres.confidence_interval.low)
    iqr_vals_mlpf_old_high.append(bsres.confidence_interval.high)
    
    bsres = scipy.stats.bootstrap((awkward.to_numpy(response_mlpf_new), ), compute_iqr)
    iqr_vals_mlpf_new_low.append(bsres.confidence_interval.low)
    iqr_vals_mlpf_new_high.append(bsres.confidence_interval.high)
    
    plt.figure()
    ax = plt.axes()
    plt.plot([], [])
    plt.hist(
        response_pf,
        bins=b,
        histtype="step", lw=2,
        label="PF: ${:.2f}\pm{:.2f}$".format(med, iqr)
    );

    med, iqr = plot_utils.med_iqr(response_mlpf_old)
    med_vals_mlpf_old.append(med)
    iqr_vals_mlpf_old.append(iqr)

    plt.hist(
        response_mlpf_old,
        bins=b,
        histtype="step", lw=2,
        label="MLPF old: ${:.2f}\pm{:.2f}$".format(med, iqr)
    );

    med, iqr = plot_utils.med_iqr(response_mlpf_new)
    med_vals_mlpf_new.append(med)
    iqr_vals_mlpf_new.append(iqr)

    plt.hist(
        response_mlpf_new,
        bins=b,
        histtype="step", lw=2,
        label="MLPF new: ${:.2f}\pm{:.2f}$".format(med, iqr)
    );
    
    plt.legend(loc=1, title="AK4 PUPPI jets, ${} < p_T < {}$".format(min_pt, max_pt))
    plt.ylim(0, 2*ax.get_ylim()[1])
    cms_label(ax)
    plt.xlabel("Jet $p_T$ response, $r=p_{T,reco}/p_{T,gen}$")
    plt.savefig("{}/ak4_puppi_jet_response_bin_{}.pdf".format(outpath, ibin))

In [ ]:
fig = plt.figure()
ax = plt.axes()
plt.plot([], [])
plt.plot(jet_bins[:-1], np.array(med_vals_pf), marker="o", label="PF")
plt.plot(jet_bins[:-1], np.array(med_vals_mlpf_old), marker="v", label="MLPF old")
plt.plot(jet_bins[:-1], np.array(med_vals_mlpf_new), marker="^", label="MLPF new")
plt.xscale("log")
plt.legend()
plt.ylabel("jet $p_T$ response median")
cms_label(ax)
ax.text(0.01, 0.95, "AK4 PUPPI jets", transform=ax.transAxes)
plt.axhline(1.0, color="black", ls="--")
plt.ylim(0.5, 1.5)
plt.savefig("{}/ak4_puppi_jet_response_median.pdf".format(outpath))

In [ ]:
fig = plt.figure()
ax = plt.axes()
plt.plot([], [])

p0 = plt.plot(
    jet_bins[:-1],
    np.array(iqr_vals_pf)/np.array(med_vals_pf),
    label="PF")
plt.fill_between(jet_bins[:-1],
    np.array(iqr_vals_pf_low)/np.array(med_vals_pf),
    np.array(iqr_vals_pf_high)/np.array(med_vals_pf),
    color=p0[0].get_color(), alpha=0.5
)

p0 = plt.plot(
    jet_bins[:-1],
    np.array(iqr_vals_mlpf_old)/np.array(med_vals_mlpf_old),
    label="MLPF old")
plt.fill_between(jet_bins[:-1],
    np.array(iqr_vals_mlpf_old_low)/np.array(med_vals_mlpf_old),
    np.array(iqr_vals_mlpf_old_high)/np.array(med_vals_mlpf_old),
    color=p0[0].get_color(), alpha=0.5
)

p0 = plt.plot(
    jet_bins[:-1],
    np.array(iqr_vals_mlpf_new)/np.array(med_vals_mlpf_new),
    label="MLPF new")
plt.fill_between(jet_bins[:-1],
    np.array(iqr_vals_mlpf_new_low)/np.array(med_vals_mlpf_new),
    np.array(iqr_vals_mlpf_new_high)/np.array(med_vals_mlpf_new),
    color=p0[0].get_color(), alpha=0.5
)

plt.xscale("log")
cms_label(ax)
plt.legend()
plt.ylabel("jet $p_T$ response IQR / median")
ax.text(0.01, 0.95, "AK4 PUPPI jets", transform=ax.transAxes)
plt.ylim(0,0.5)
plt.savefig("{}/ak4_puppi_jet_response_iqr_over_median.pdf".format(outpath))

In [ ]:
f, (a0, a1) = plt.subplots(2, 1, gridspec_kw={"height_ratios": [3, 1]}, sharex=True)

h0 = to_bh(gen_met_pt, met_bins)
h1 = to_bh(pf_met_pt, met_bins)
h2 = to_bh(mlpf_old_met_pt, met_bins)
h3 = to_bh(mlpf_new_met_pt, met_bins)

plt.sca(a0)
x0 = mplhep.histplot(h0, histtype="step", lw=2, label="gen", binwnorm=1.0, ls="--")
x1 = mplhep.histplot(h1, histtype="step", lw=2, label="PF", binwnorm=1.0, ls="-")
x2 = mplhep.histplot(h2, histtype="step", lw=2, label="MLPF old", binwnorm=1.0, ls="-")
x3 = mplhep.histplot(h3, histtype="step", lw=2, label="MLPF new", binwnorm=1.0, ls="-")

# plt.xscale("log")
plt.yscale("log")
cms_label(a0)
# sample_label(a0, physics_process)
handles, labels = a0.get_legend_handles_labels()
handles = [x0[0].stairs, x1[0].stairs, x2[0].stairs, x3[0].stairs]

a0.legend(handles, labels, loc=1)
plt.ylim(1, 10**5)
plt.ylabel("Number of events / bin")

plt.sca(a1)
mplhep.histplot(h0 / h0, histtype="step", lw=2, ls="--")
mplhep.histplot(h1 / h0, histtype="step", lw=2, ls="-")
mplhep.histplot(h2 / h0, histtype="step", lw=2, ls="-")
mplhep.histplot(h3 / h0, histtype="step", lw=2, ls="-")
if folder == "QCD_PU":
    plt.ylim(-10,100)
elif folder == "TTbar_PU":
    plt.ylim(-2,10)

plt.ylabel("reco / gen")
plt.xlabel("MET [GeV]")
plt.xlim(min(met_bins), max(met_bins))

plt.savefig("{}/met.pdf".format(outpath))

In [ ]:
pf_met_response = pf_met_pt/gen_met_pt
mlpf_old_met_response = mlpf_old_met_pt/gen_met_pt
mlpf_new_met_response = mlpf_new_met_pt/gen_met_pt

In [ ]:
plt.plot([], [])
plt.hist(pf_met_pt[gen_met_pt<1], bins=np.linspace(0, 500, 101), histtype="step", lw=2, label="PF");
plt.hist(mlpf_old_met_pt[gen_met_pt<1], bins=np.linspace(0, 500, 101), histtype="step", lw=2, label="MLPF old");
plt.hist(mlpf_new_met_pt[gen_met_pt<1], bins=np.linspace(0, 500, 101), histtype="step", lw=2, label="MLPF new");
plt.yscale("log")
plt.legend(loc="best")

In [ ]:
plt.plot([], [])
plt.hist(pf_met_response[gen_met_pt>1], bins=np.linspace(0, 10, 41), histtype="step", lw=2, label="PF");
plt.hist(mlpf_old_met_response[gen_met_pt>1], bins=np.linspace(0, 10, 41), histtype="step", lw=2, label="MLPF old");
plt.hist(mlpf_new_met_response[gen_met_pt>1], bins=np.linspace(0, 10, 41), histtype="step", lw=2, label="MLPF new");
#plt.yscale("log")
plt.legend(loc="best")

In [ ]:
# k = "DQMData/Run 1/JetMET/Run summary/Jet/CleanedslimmedJetsPuppi/Pt"
# hi1 = get_hist_and_merge(files1, k)
# hi2 = get_hist_and_merge(files2, k)

# ax = plt.axes()
# mplhep.histplot(hi1, label="PF")
# mplhep.histplot(hi2, label="MLPF")
# # plt.axhline(1.0, color="black")
# plt.legend(loc=(0.75, 0.8))
# cms_label(ax)
# plt.xlabel("Jet $p_T$ [GeV]")
# plt.ylabel("Number of jets")
# plt.ylim(1e1, 1e6)
# plt.yscale("log")
# plt.savefig("cmssw/jet_pt_{}.pdf".format(physics_process), bbox_inches="tight")

In [ ]:
# k = "DQMData/Run 1/JetMET/Run summary/Jet/Uncleanedak4PFJets/Eta"
# hi1 = get_hist_and_merge(files1, k)
# hi2 = get_hist_and_merge(files2, k)

# ax = plt.axes()
# mplhep.histplot(hi1, label="PF")
# mplhep.histplot(hi2, label="MLPF")
# # plt.axhline(1.0, color="black")
# plt.legend(loc=(0.75, 0.8))
# plt.ylim(bottom=10, top=1e5)
# cms_label(ax)
# plt.xlabel("Jet $\eta$")
# plt.ylabel("Number of jets")
# plt.yscale("log")
# plt.savefig("cmssw/jet_eta_{}.pdf".format(physics_process))

In [ ]:
# for k in uproot.open(files1[0]).keys():
#     if "DQMData/Run 1/ParticleFlow" in k:
#         print(k)

In [ ]:
# k = "DQMData/Run 1/JetMET/Run summary/MET/pfMet/Cleaned/MET"
# hi1 = get_hist_and_merge(files1, k)
# hi2 = get_hist_and_merge(files2, k)

# ax = plt.axes()
# mplhep.histplot(hi1, label="PF")
# mplhep.histplot(hi2, label="MLPF")
# # plt.axhline(1.0, color="black")
# plt.legend(loc=(0.75, 0.7))
# cms_label(ax)
# plt.xlabel("MET [GeV]")
# plt.ylabel("Number of events")
# plt.yscale("log")
# plt.ylim(1, 1e7)
# plt.savefig("cmssw/met_{}.pdf".format(physics_process))

In [ ]:
# hi1 = get_hist_and_merge(
#     [files1], "DQMData/Run 1/ParticleFlow/Run summary/PFMETValidation/CompWithGenMET/delta_et_Over_et_VS_et_"
# )
# hi2 = get_hist_and_merge(
#     [files2], "DQMData/Run 1/ParticleFlow/Run summary/PFMETValidation/CompWithGenMET/delta_et_Over_et_VS_et_"
# )

# met_response_pf = fit_response(hi1, range(5, 10))
# met_response_mlpf = fit_response(hi2, range(5, 10))

In [ ]:
# for k in uproot.open("/local/joosep/mlpf/results/cms/CMSSW_14_1_0_pre3/QCD_PU_pf/DQM_V0001_R000000001__Global__CMSSW_X_Y_Z__RECO.root").keys():
#     if "MET" in k:
#         print(k)

In [ ]:
# hi1 = get_hist_and_merge(
#     files1, "DQMData/Run 1/ParticleFlow/Run summary/PFJetValidation/CompWithGenJet/delta_et_Over_et_VS_et_"
# )
# hi2 = get_hist_and_merge(
#     files2, "DQMData/Run 1/ParticleFlow/Run summary/PFJetValidation/CompWithGenJet/delta_et_Over_et_VS_et_"
# )

# jet_response_pf = fit_response(hi1, range(4, 10))
# jet_response_mlpf = fit_response(hi2, range(4, 10))

In [ ]:
# fig = plt.figure()
# ax = plt.axes()

# plt.errorbar(
#     met_response_pf[0],
#     1.0 - met_response_pf[1],
#     met_response_pf[2],
#     lw=0,
#     markersize=10,
#     elinewidth=2,
#     alpha=0.8,
#     marker="o",
#     label="PF",
# )
# plt.errorbar(
#     met_response_mlpf[0],
#     1.0 - met_response_mlpf[1],
#     met_response_mlpf[2],
#     lw=0,
#     markersize=10,
#     elinewidth=2,
#     alpha=0.8,
#     marker="s",
#     label="MLPF",
# )
# # plt.xscale("log")

# plt.xlabel("GenMET $E_T$ [GeV]")
# plt.ylabel("MET response")
# plt.legend(loc=(0.75, 0.7))
# plt.xlim(0, 500)
# plt.ylim(0, 2)
# cms_label(ax)
# sample_label(ax, physics_process)
# plt.savefig("cmssw/met_response_{}.pdf".format(file_suffix))

In [ ]:
# fig = plt.figure()
# ax = plt.axes()

# plt.errorbar(
#     met_response_pf[0],
#     met_response_pf[3],
#     met_response_pf[4],
#     lw=0,
#     markersize=10,
#     elinewidth=2,
#     alpha=0.8,
#     marker="o",
#     label="PF",
# )
# plt.errorbar(
#     met_response_mlpf[0],
#     met_response_mlpf[3],
#     met_response_mlpf[4],
#     lw=0,
#     markersize=10,
#     elinewidth=2,
#     alpha=0.8,
#     marker="s",
#     label="MLPF",
# )
# # plt.xscale("log")

# plt.xlabel("GenMET $E_T$ [GeV]")
# plt.ylabel("MET resolution")
# plt.legend(loc=(0.75, 0.7))
# plt.xlim(0, 500)
# plt.ylim(0, 2)
# cms_label(ax)
# sample_label(ax, physics_process)
# plt.savefig("cmssw/met_resolution_{}.pdf".format(file_suffix))

In [ ]:
# fig = plt.figure()
# ax = plt.axes()

# plt.errorbar(
#     jet_response_pf[0],
#     1.0 - jet_response_pf[1],
#     jet_response_pf[2],
#     lw=0,
#     markersize=10,
#     elinewidth=2,
#     alpha=0.8,
#     marker="o",
#     label="PF",
# )
# plt.errorbar(
#     jet_response_mlpf[0],
#     1.0 - jet_response_mlpf[1],
#     jet_response_mlpf[2],
#     lw=0,
#     markersize=10,
#     elinewidth=2,
#     alpha=0.8,
#     marker="s",
#     label="MLPF",
# )
# # plt.xscale("log")

# plt.xlabel("GenJet $E_T$ [GeV]")
# plt.ylabel("Jet response")
# plt.legend(loc=(0.75, 0.7))
# plt.xlim(0, 500)
# plt.ylim(0, 2)
# cms_label(ax)
# sample_label(ax, physics_process)
# plt.savefig("cmssw/jet_response_{}.pdf".format(file_suffix))

In [ ]:
# fig = plt.figure()
# ax = plt.axes()

# plt.errorbar(
#     jet_response_pf[0],
#     jet_response_pf[3],
#     jet_response_pf[4],
#     lw=0,
#     markersize=10,
#     elinewidth=2,
#     alpha=0.8,
#     marker="o",
#     label="PF",
# )
# plt.errorbar(
#     jet_response_mlpf[0],
#     jet_response_mlpf[3],
#     jet_response_mlpf[4],
#     lw=0,
#     markersize=10,
#     elinewidth=2,
#     alpha=0.8,
#     marker="s",
#     label="MLPF",
# )
# # plt.xscale("log")

# plt.xlabel("GenJet $E_T$ [GeV]")
# plt.ylabel("Jet resolution")
# plt.legend(loc=(0.75, 0.7))
# plt.xlim(0, 500)
# plt.ylim(0, 1)
# cms_label(ax)
# sample_label(ax, physics_process)
# plt.savefig("cmssw/jet_resolution_{}.pdf".format(file_suffix))